In [1]:
# !pip install pyaudio torchaudio librosa transformers

In [2]:
from transformers import WhisperProcessor,WhisperFeatureExtractor, WhisperTokenizer,  WhisperForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
import numpy as np
import torchaudio
from IPython.display import Audio as display_Audio, display
import librosa

C:\Users\Megha-TechUnicorn\anaconda3\envs\payoneer\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [53]:

def eval_audio(audio_sample):
    # audio,samplerate =  librosa.load(audio_sample)   
    samplerate=16000
    resample= librosa.resample(audio_sample, orig_sr= samplerate, target_sr=16000)
    # print(resample)
    inp_features= extractor(np.array(audio_sample), sampling_rate=16000, return_tensors='pt').input_features 
    inp_features =inp_features.to('cuda:1')
    model.to( 'cuda:1')
    x= model.generate(inp_features)
    return processor.tokenizer.batch_decode(x, skip_special_tokens=True)


def audio_tester(filename):

    # resample the audio and generate chunks
    # audio,samplerate =  librosa.load(filename) 
    audio= filename
    samplerate = 16000
    audio= librosa.effects.preemphasis(audio)
    x = librosa.effects.split(audio, top_db=40)
    display(display_Audio(audio, rate= samplerate))
    print("number of chunkx", len(x))
    audio_samples= []
    for i, interval in enumerate(x):
        start, end= interval
        # print(start, end)
        audio_samples.append(audio[start:end])

    final_speech =""
    for a in audio_samples:
        if librosa.get_duration(y=a, sr=samplerate)>0:
            chunked_audio = sample_maker(a,  samplerate)
            for c in chunked_audio:
                # clip= display_Audio(c, rate= samplerate)
                # display(clip)
     
                res= eval_audio(c)[0]+ " "
                print(res)
                final_speech+= res
    return final_speech

# aud, sr = librosa.load('/kaggle/input/deep-voice-deepfake-voice-recognition/KAGGLE/AUDIO/REAL/biden-original.wav')
def sample_maker(aud,sr=16000):
    chunk_duration= 13*sr
    total= len(aud)
    samples_created= 0
     
    split_samples= []
    
    
    while samples_created < total:
        if chunk_duration > total - samples_created:
            chunk_duration=  total - samples_created
        split_samples.append(aud[samples_created: chunk_duration +samples_created+2])
        samples_created+= chunk_duration
    return split_samples

In [4]:
 # for generating the input features of the audio 
extractor= WhisperFeatureExtractor.from_pretrained("openai/whisper-base" )

# tokenizing and padding the transcription text in the train and val se
tokenizer= WhisperTokenizer.from_pretrained("openai/whisper-base", task= 'transcribe', language='English')
model = WhisperForConditionalGeneration.from_pretrained("meghaverse/outputs")

def prep_data(batch):
    audio= batch['audio']
    batch['input_features']= extractor(audio['array'], sampling_rate= audio['sampling_rate']).input_features[0]
    batch['labels']= tokenizer(batch['text']).input_ids
    return batch


In [35]:
import pyaudio as pa

py_audio = pa.PyAudio()
print("device information")
for i in range(5):
    print(py_audio.get_device_info_by_index(i)['name'])

py_audio.get_device_info_by_index(2) 

device information
Microsoft Sound Mapper - Input
Microphone (Realtek(R) Audio)
Microphone Array (IntelÂ® Smart 
Microsoft Sound Mapper - Output
Realtek HD Audio 2nd output (Re


{'index': 2,
 'structVersion': 2,
 'name': 'Microphone Array (IntelÂ® Smart ',
 'hostApi': 0,
 'maxInputChannels': 4,
 'maxOutputChannels': 0,
 'defaultLowInputLatency': 0.09,
 'defaultLowOutputLatency': 0.09,
 'defaultHighInputLatency': 0.18,
 'defaultHighOutputLatency': 0.18,
 'defaultSampleRate': 44100.0}

In [58]:
import os
import keyboard
import time
import math
import wave 
CHUNK = 1024
FORMAT = pa.paInt16
CHANNELS = 2
RATE = 44100
DURATION = 10
 
def convert_byte_to_arr(buf):
    join_frames= b''.join(buf)
    np_arr= np.frombuffer(join_frames, dtype= np.float32)
    tetx = audio_tester(np_arr)
    return tetx
     
p = pa.PyAudio()

stream= p.Stream(p,
    format= FORMAT,
    channels= CHANNELS,
    rate = RATE,
    input= True,
    input_device_index= 1,
    frames_per_buffer=CHUNK
)

frames= []

while True:
    data = stream.read(CHUNK)
    frames.append(data)
    
    if keyboard.is_pressed('q') :
        stream.stop_stream()
        break
if stream.is_stopped():
    stream.close()
    p.terminate()

convert_byte_to_arr(frames)


C:\Users\Megha-TechUnicorn\anaconda3\envs\payoneer\lib\site-packages\librosa\util\utils.py:2501: RuntimeWarning: overflow encountered in power
  return np.power(x, 2, dtype=dtype)  # type: ignore


number of chunkx 0


''

In [54]:
print(len(frames))

joined_frames= b''.join(frames)
# m = bytearray(joined_frames)
d = np.frombuffer(joined_frames, dtype= np.float32)
audio_tester(d)

149


C:\Users\Megha-TechUnicorn\anaconda3\envs\payoneer\lib\site-packages\IPython\lib\display.py:188: RuntimeWarning: invalid value encountered in cast
  return scaled.astype("<h").tobytes(), nchan


number of chunkx 0


''

In [ ]:
import pyaudio
def record(outputFile):
    CHUNK = 1024
    FORMAT = pyaudio.paInt16
    CHANNELS = 2
    RATE = 44100
    DURATION = 5

    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)

    print("Recording....")
    frames = []

    while stream.is_active():
        
        data = stream.read(CHUNK)
        frames.append(data)
        if keyboard.is_pressed('q') == 'q':
             stream.stop_stream()
             stream.close()
    p.terminate()

    wf = wave.open(outputFile, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()

record('output1.wav')